In [4]:
import redis
import csv
import numpy as np
from sentence_transformers import SentenceTransformer
from redis.commands.search.query import Query
from redis.commands.search.field import TextField, TagField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
import openai
import tiktoken

VSS_MINIMUM_SCORE=2

/Users/mortensi/venvs/ps/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
conn = redis.Redis(host='localhost', port=6380, decode_responses=True)
print(conn.ping())

True


In [16]:
def load():
    with open("imdb_movies.csv", encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        cnt = 0
        for row in csvReader:
            conn.json().set(f'moviebot:movie:{cnt}', '$', row)
            cnt = cnt + 1
        print("Data was loaded")

In [24]:
def create_index():
    indexes = conn.execute_command("FT._LIST")
    print(indexes)
    if "movie_idx" not in indexes:
        index_def = IndexDefinition(prefix=["moviebot:movie:"], index_type=IndexType.JSON)
        schema = (TextField("$.crew", as_name="crew"),
                  TextField("$.overview", as_name="overview"),
                  TagField("$.genre", as_name="genre"),
                  TagField("$.names", as_name="names"),
                  VectorField("$.overview_embedding", "HNSW", {"TYPE": "FLOAT32", "DIM": 384, "DISTANCE_METRIC": "L2"}, as_name="embedding"))
        conn.ft('movie_idx').create_index(schema, definition=index_def)
        print("The index has been created")
    else:
        print("The index exists")

In [34]:
def create_embeddings():
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    for key in conn.scan_iter(match='moviebot:movie:*'):
        print(f"creating the embedding for {key}")
        result = conn.json().get(key, "$.names", "$.overview", "$.crew", "$.score", "$.genre")
        movie = f"movie title is: {result['$.names'][0]}\n"
        movie += f"movie genre is: {result['$.genre'][0]}\n"
        movie += f"movie crew is: {result['$.crew'][0]}\n"
        movie += f"movie score is: {result['$.score'][0]}\n"
        movie += f"movie overview is: {result['$.overview'][0]}\n"
        movie += f"movie overview is: {result['$.overview'][0]}\n"
        conn.json().set(key, "$.overview_embedding", model.encode(movie).astype(np.float32).tolist())

In [5]:
def get_prompt(model, query):
    context = ""
    prompt = ""
    q = Query("@embedding:[VECTOR_RANGE $radius $vec]=>{$YIELD_DISTANCE_AS: score}") \
        .sort_by("score", asc=True) \
        .return_fields("overview", "names", "score", "$.crew", "$.genre", "$.score") \
        .paging(0, 3) \
        .dialect(2)

    # Find all vectors within VSS_MINIMUM_SCORE of the query vector
    query_params = {
        "radius": VSS_MINIMUM_SCORE,
        "vec": model.encode(query).astype(np.float32).tobytes()
    }

    res = conn.ft("movie_idx").search(q, query_params)

    if (res is not None) and len(res.docs):
        it = iter(res.docs[0:])
        for x in it:
            # print("the score is: " + str(x['score']))
            movie = f"movie title is: {x['names']}\n"
            movie += f"movie genre is: {x['$.genre']}\n"
            movie += f"movie crew is: {x['$.crew']}\n"
            movie += f"movie score is: {x['$.score']}\n"
            movie += f"movie overview is: {x['overview']}\n"
            context += movie + "\n"

    if len(context) > 0:
        prompt = '''Use the provided information to answer the search query the user has sent.
            The information in the database provides three movies, chose the one or the ones that fit most.
            If you can't answer the user's question, say "Sorry, I am unable to answer the question, try to refine your question". Do not guess. You must deduce the answer exclusively from the information provided. 
            The answer must be formatted in markdown or HTML.
            Do not make things up. Do not add personal opinions. Do not add any disclaimer.

            Search query: 

            {}

            Information in the database: 

            {}
            '''.format(query, context)

    return prompt

In [14]:
def getOpenAIGPT35(prompt):
    # Define the system message
    system_msg = 'You are a smart and knowledgeable AI assistant with an expertise in all kind of movies. You are a very friendly and helpful AI. You are empowered to recommend movies based on the provided context. Do NOT make anything up. Do NOT engage in topics that are not about movies.';

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0613")
    # print("tokens: " + str(num_tokens_from_string(prompt, "cl100k_base")))

    try:
        response = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613",
                                                stream=False,
                                                messages=[{"role": "system", "content": system_msg},
                                                          {"role": "user", "content": prompt}])
        return response["choices"][0]["message"]["content"]
    except openai.error.OpenAIError as e:
        # Handle the error here
        if "context window is too large" in str(e):
            print("Error: Maximum context length exceeded. Please shorten your input.")
            return "Maximum context length exceeded"
        else:
            print("An unexpected error occurred:", e)
            return "An unexpected error occurred"


def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [15]:
def render():
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    # React to user input
    while True:
        question = input("Ask a question")
        reply = f"You asked: {question}"
        prompt = get_prompt(model, question)
        response = getOpenAIGPT35(prompt)
        print(response)
        print("--------------------------------")

In [ ]:
render()

Ask a question recommend an italian movie


I recommend you watch **La Dolce Vita**. It is an Italian comedy-drama film starring Marcello Mastroianni. The movie follows the story of a journalist named Marcello who struggles to find his place in the world, torn between the allure of Rome's elite social scene and the stifling domesticity offered by his girlfriend, all the while searching for a way to become a serious writer. With a score of 82.0, it is highly regarded and explores themes of love, desire, and existentialism in mid-century Italy. Enjoy!
--------------------------------


Ask a question raccomandami un film italiano


Based on the information in the database, I recommend the movie "La Dolce Vita." 

- Movie title: La Dolce Vita
- Genre: Comedy, Drama
- Crew: Marcello Mastroianni, Anita Ekberg, Anouk Aimée, Yvonne Furneaux, Magali Noël, and more
- Score: 82.0

Overview: Journalist and man-about-town Marcello struggles to find his place in the world, torn between the allure of Rome's elite social scene and the stifling domesticity offered by his girlfriend, all the while searching for a way to become a serious writer.

Enjoy the movie!
--------------------------------


Ask a question raccomandami un film italiano e rispondi in italiano


Mi dispiace, non posso rispondere in italiano. Tuttavia, posso consigliarti alcuni film italiani in base alle informazioni fornite:

1. How Funny Can Sex Be? (57.0): Questo film è una raccolta di nove episodi che affronta il tema dell'amore, del sesso e del matrimonio nella contemporaneità dell'Italia degli anni '70.

2. La Dolce Vita (82.0): Un giornalista, Marcello, si trova in bilico tra la scena sociale élite di Roma e la vita domestica proposta dalla sua fidanzata, mentre cerca di trovare il suo posto nel mondo e diventare uno scrittore serio.

Spero che queste raccomandazioni siano di tuo interesse!
--------------------------------


Ask a question chi è il direttore di Interstellar?


The movie "The Science of Interstellar" is a documentary about Christopher Nolan's sci-fi film "Interstellar". While the movie focuses on the scientific foundations, themes, and visuals of "Interstellar", it does not mention the director of the film. Therefore, based on the information provided, I am unable to answer the question about the director of "Interstellar".
--------------------------------


Ask a question what is the score of interstellar?


The score of "Interstellar" is 84.0.
--------------------------------
